In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import pandas as pd
import re

In [25]:
def setup_driver():
    brave_path = r"C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe"
    service = Service(ChromeDriverManager().install())
    options = Options()
    options.binary_location = brave_path
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [26]:
def get_page_source(url, driver):
    driver.get(url)
    time.sleep(10)
    wait = WebDriverWait(driver, 10)
    return driver.page_source

In [27]:
def extract_articles_jornal_noticias(soup):

    # divs_itc001 = soup.find_all("div", class_=re.compile(r"\bITC001\b"))
    #divs_hc002 = soup.find_all("div", class_=re.compile(r"\bHC002\b"))
    # divs_pc001 = soup.find_all("div", class_=re.compile(r"\bPC001\b"))
    # divs_hc003 = soup.find_all("div", class_=re.compile(r"\bHC003\b"))
    # divs_itc002 = soup.find_all("div", class_=re.compile(r"\bITC002\b"))
    divs_hc002data = soup.find_all(lambda tag: (
        tag.name == "div" and 
        tag.get("data-instance") == "i02" and 
        "HC002" in tag.get("class", []) and 
        "light" in tag.get("class", [])
    ))

    
    print(f"HC002 light none found: {len(divs_hc002data)}")
    # print(f"ITC001 light none found: {len(divs_itc001)}")
    # print(f"PC001 light corners-square none hover-scale found: {len(divs_pc001)}")
    # print(f"HC003 light corners-square none found: {len(divs_hc003)}")
    # print(f"ITC002 light corners-square none found: {len(divs_itc002)}")
    #print(f"HC002 light none found: {len(divs_hc002data)}")

    #all_divs = divs_hc002 + divs_itc001 + divs_pc001 +  divs_hc003 +  divs_itc002 + divs_hc002data
    all_divs = divs_hc002data

    articles = []
    for div in all_divs:
        print(div)
        a_tag = div.find("a")
        h2title, category = None, None
        h2title = div.find("h2",class_="title")
        category = div.find("div", class_="info")

        if a_tag:

            h2title = h2title.get_text(strip=True)
            category = category.get_text(strip=True)
            link = a_tag["href"]
            
            full_link = f"https://www.jn.pt{link}"  # Ensure full URL

            articles.append({"title": h2title, "link": full_link, "category": category, "div": div})

        else:
            continue
            
    return articles

In [28]:
def extract_full_article(article_url, driver):
    """Visit an article URL and extract the full text content."""
    page_source = get_page_source(article_url, driver)
    soup = BeautifulSoup(page_source, "html.parser")

    paragraphs = soup.find_all("p")
    return "\n".join([p.get_text(strip=True) for p in paragraphs])

In [32]:
def main():
    url = "https://www.jn.pt/"
    driver = setup_driver()

    get_page_source(url, driver)
    driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[2]/div[2]/div[2]/button[1]').click()
    time.sleep(1)
    
    total_height = driver.execute_script("return document.body.scrollHeight")
    quarter_height = total_height / 4
    current_scroll_position = 0

    for _ in range(4):  # Scroll four times for 25%, 50%, 75%, 100%
        current_scroll_position += quarter_height
        driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
        time.sleep(5) #Add a small delay, adjust as needed

    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    articles = extract_articles_jornal_noticias(soup)

    # for article in articles:
    #     article["article_content"] = extract_full_article(article["link"], driver)

    #     print(f"Title: {article['title']}")
    #     print(f"Link: {article['link']}")
    #     print(f"Content Inside Div: {article['div_content']}")
    #     print(f"Full Article Content:\n{article['article_content'][:500]}...")
    #     print("-" * 100)

    driver.quit()

    df = pd.DataFrame(articles)

    df = df.drop_duplicates(subset=['title'],ignore_index=True)

    display(df)
    
if __name__ == "__main__":
    main()


HC002 light none found: 11
<div class="HC002 light state01" data-instance="i02"><a class="clean-link" cmp-ltrk="Home - +JN" cmp-ltrk-idx="1" data-mrf-link="https://www.jn.pt/1484730721/matosinhos-quer-inverter-taxa-de-abandono-escolar/" href="/1484730721/matosinhos-quer-inverter-taxa-de-abandono-escolar/" mrfobservableid="3b6f2463-5abe-4512-ae62-b09be7fa8fb4" target=""><div class="media corners-square"><div class="embed-responsive embed-responsive-3by2"><picture><!-- --> <!-- --> <!-- --> <!-- --> <!-- --> <!-- --> <!-- --> <!-- --> <!-- --> <!-- --> <source data-srcset="https://asset.skoiy.com/jnrcnjwljorqgsbu/u2auevckrifr.jpg?w=240&amp;q=90&amp;" media="(max-width: 575px)" srcset="https://asset.skoiy.com/jnrcnjwljorqgsbu/u2auevckrifr.jpg?w=240&amp;q=90&amp;"/> <source data-srcset="https://asset.skoiy.com/jnrcnjwljorqgsbu/u2auevckrifr.jpg?w=275&amp;q=90&amp;" media="(max-width: 767px)" srcset="https://asset.skoiy.com/jnrcnjwljorqgsbu/u2auevckrifr.jpg?w=275&amp;q=90&amp;"/> <source dat

,title,link,category,div
0,Matosinhos quer inverter taxa de abandono escolar,https://www.jn.pt/1484730721/matosinhos-quer-i...,exclusivoEducação,"[[[<div class=""embed-responsive embed-responsi..."
1,"Portugal de olho nas ""meias"" num estádio onde ...",https://www.jn.pt/6291649738/portugal-de-olho-...,exclusivoCopenhaga,"[[[<div class=""embed-responsive embed-responsi..."
2,Este é o mês de março com mais depressões mas ...,https://www.jn.pt/5890673096/este-e-o-mes-de-m...,exclusivoMau tempo,"[[[<div class=""embed-responsive embed-responsi..."
3,Último troço da Avenida até ao Mar deve arranc...,https://www.jn.pt/3261065157/ultimo-troco-da-a...,Mobilidade,"[[[<div class=""embed-responsive embed-responsi..."
4,Câmara de Matosinhos prevê instalar cerca de 4...,https://www.jn.pt/4996927658/camara-de-matosin...,Segurança,"[[[<div class=""embed-responsive embed-responsi..."
5,Realizador Carl Erik Rinsch acusado de roubar ...,https://www.jn.pt/7178363392/realizador-carl-e...,Fraude,"[[[<div class=""embed-responsive embed-responsi..."
6,Marcelo dissolve Parlamento e convoca eleições...,https://www.jn.pt/2685189158/marcelo-dissolve-...,Política,"[[[<div class=""embed-responsive embed-responsi..."
7,"Depois de Espanha, Portugal também vai debater...",https://www.jn.pt/1079628399/depois-de-espanha...,Segurança rodoviária,"[[[<div class=""embed-responsive embed-responsi..."
8,Academia de Notícias: o que precisa de saber,https://www.jn.pt/3340995480/academia-de-notic...,Média,"[[[<div class=""embed-responsive embed-responsi..."
9,O que muda na entrega do IRS?,https://www.jn.pt/8483550877/o-que-muda-na-ent...,Impostos,"[[[<div class=""embed-responsive embed-responsi..."
